# Распределенное обучение классических моделей

Поговорим про то, как решать задачу машинного обучения, когда самих наблюдений очень много и они все не помещаются на машину.

Центральная идея во всех алгоритмах - параллельно на нескольких машинах посчитать частичные элементы, которые требуются для принятия решения, передать их на центральную машину и сделать шаг алгоритма.

Для обучения линейных моделей на различных машинах будем считать градиент и на главной машине делать шаг градиентного спуска.

Для деревьев решений на различных машинах будем считать распределение по корзинкам (по бинам) и на главной машине будет определять порог для определенного признака.

### Распределенное обучение VW

Vowpal Wabbit также умеет работать распределенно, что делает его универсальным инструментом для обучения линейных моделей на больших данных. Для работы он использует дополнительный компонент - `spanning_tree` - это специальный процесс, который координирует работу различных воркеров между собой.

Про него можно также думать, как про корневую вершину в алгоритме "Tree Allreduce", который используется для эффективной утилизации сети при обучении.

Чтобы иметь возможность использовать `spanning_tree`, необходимо собрать VW руками.


Собирем VW. Делать это нужно с суперпользоателя, поэтому удобнее всего запускать из терминала.

```bash
apt update && \
apt install git psmisc -y && \
apt install libboost-dev libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-math-dev libboost-test-dev zlib1g-dev cmake g++ -y 


wget https://github.com/google/flatbuffers/archive/v1.12.0.tar.gz && \
tar -xzf v1.12.0.tar.gz && \
cd flatbuffers-1.12.0 && \
mkdir build_dir && \
cd build_dir && \
cmake -G "Unix Makefiles" -DFLATBUFFERS_BUILD_TESTS=Off -DFLATBUFFERS_INSTALL=On -DCMAKE_BUILD_TYPE=Release DFLATBUFFERS_BUILD_FLATHASH=Off .. && \
make install -j$(nproc) && \
cd ../..

git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git && \
cd vowpal_wabbit && \
sudo make && \
cd build && \
sudo make install -j$(nproc)
```

**Хозяйке на заметку** Чтобы получить рутовый доступ с кластера в Azure через Jupyter можно открыть терминал и по ssh подключиться к пользователю `azureuser`. Текущий пользователь `spark` к сожалению имеет очень мало прав.

```bash
ssh azureuser@localhost
sudo su
```

In [ ]:
%%writefile install_vw.sh

sudo apt update -y
sudo apt install git psmisc -y 
sudo apt install libboost-dev libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-math-dev libboost-test-dev zlib1g-dev cmake g++ -y 

wget https://github.com/google/flatbuffers/archive/v1.12.0.tar.gz && \
    tar -xzf v1.12.0.tar.gz && \
    cd flatbuffers-1.12.0 && \
    mkdir build_dir && \
    cd build_dir && \
    cmake -G "Unix Makefiles" -DFLATBUFFERS_BUILD_TESTS=Off -DFLATBUFFERS_INSTALL=On -DCMAKE_BUILD_TYPE=Release DFLATBUFFERS_BUILD_FLATHASH=Off .. && \
    make install -j$(nproc) && \
    cd ../..
    
git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git && \
    cd vowpal_wabbit && \
    git checkout d1ead9a0a9afd56d2ee11a72e0c1aaa7702ee281 && \
    sudo make && \
    cd build && \
    sudo make install -j$(nproc)

In [ ]:
! bash install_vw.sh

In [ ]:
! which vw

In [ ]:
! which spanning_tree

In [ ]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
! unzip drugsCom_raw.zip

In [ ]:
! hdfs dfs -ls /user

In [ ]:
! hdfs dfs -rm -r /user/drugs/data || true
! hdfs dfs -mkdir -p /user/drugs/data

In [ ]:
! hdfs dfs -ls /user/drugs

Выгрузим датасет с препаратами.

In [ ]:
%%bash

cat drugsComTrain_raw.tsv <(tail -n +2 drugsComTest_raw.tsv) | hdfs dfs -put - /user/drugs/data/drugs.tsv

In [ ]:
! hdfs dfs -ls  -h /user/drugs/data

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
sc = pyspark.SparkContext(appName="lsml-app-1")

In [ ]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [ ]:
from pyspark.sql import functions as F
from datetime import datetime
import re

In [13]:
data = se.read.option("delimiter", "\t").csv('/user/drugs/data/*', header=True, inferSchema=True)

In [15]:
data.limit(10).toPandas()

,_c0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""""""It has no side effect, I take it in combina...",9.0,"May 20, 2012",27.0
1,95260,Guanfacine,ADHD,"""""""My son is halfway through his fourth week o...",None,None,NaN
2,We have tried many different medications and s...,8.0,"April 27, 2010",192,None,None,NaN
3,92703,Lybrel,Birth Control,"""""""I used to take another oral contraceptive, ...",None,None,NaN
4,The positive side is that I didn&#039;t have a...,5.0,"December 14, 2009",17,None,None,NaN
5,138000,Ortho Evra,Birth Control,"""""""This is my first time using any form of bir...",8.0,"November 3, 2015",10.0
6,35696,Buprenorphine / naloxone,Opiate Dependence,"""""""Suboxone has completely turned my life arou...",9.0,"November 27, 2016",37.0
7,155963,Cialis,Benign Prostatic Hyperplasia,"""""""2nd day on 5mg started to work with rock ha...",2.0,"November 28, 2015",43.0
8,165907,Levonorgestrel,Emergency Contraception,"""""""He pulled out, but he cummed a bit in me. I...",1.0,"March 7, 2017",5.0
9,102654,Aripiprazole,Bipolar Disorde,"""""""Abilify changed my life. There is hope. I w...",10.0,"March 14, 2015",32.0


Мы будем запускать 2 воркера. Поэтмоу разделим весь датасет на 3 части - 2 равные для воркером и 1 маленькую часть для теста.

In [16]:
part1, part2, test = (
    data
    .na.drop('any')
    .randomSplit([0.45, 0.45, 0.1], 422)
)

Соберем датасет на спарке

In [17]:
def convert_to_vw(data):
    target = data['usefulCount']
    
    drug_name = data['drugName'].lower().replace(' ', '_')
    condition = data['condition'].lower().replace(' ', '_')
    
    raw_text = data['review'].lower()
    word_pattern = re.compile(r"[a-zA-Z0-9_]+")
    words = [match.group(0) for match in re.finditer(word_pattern, raw_text)]
    review = ' '.join(words)
    
    rating = data['rating']
    
    weekday = datetime.strptime(data['date'], '%B %d, %Y').weekday()
    
    template = "{target} |d {drug_name} |c {condition} |r {review} |w {weekday} |s rating:{rating}"
    return template.format(
        target=target,
        drug_name=drug_name,
        condition=condition,
        review=review,
        weekday=weekday,
        rating=rating
    )

In [18]:
! hdfs dfs -rm -r /user/drugs/*.vw

Deleted /user/drugs/part1.vw
Deleted /user/drugs/part2.vw
Deleted /user/drugs/test.vw


In [19]:
part1.rdd.map(convert_to_vw).saveAsTextFile('/user/drugs/part1.vw')
part2.rdd.map(convert_to_vw).saveAsTextFile('/user/drugs/part2.vw')
test.rdd.map(convert_to_vw).saveAsTextFile('/user/drugs/test.vw')

In [20]:

! hdfs dfs -cat /user/drugs/part1.vw/* > train.part1.vw
! hdfs dfs -cat /user/drugs/part2.vw/* > train.part2.vw
! hdfs dfs -cat /user/drugs/test.vw/* > test.vw

Посмотрим, какие результаты мы получим, если просто запустим VW на всем файле.

In [21]:
! cat train.*.vw > train.full.vw

In [22]:
import numpy as np
from sklearn.metrics import r2_score


def calc_r2(predictions_filename, answers_filename):
    def read_target_from_vw(vw_record):
        return float(vw_record.split(' ')[0])
    
    with open(predictions_filename, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_filename, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [23]:
! vw --help | head

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value


Обучаем VW на одном файле целиком

In [24]:
%%time

! vw --final_regressor drugs.model.bin train.full.vw \
    --onethread \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
final_regressor = drugs.model.bin
Num weight bits = 23
learning rate = 20
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.full.vw.cache
Reading datafile = train.full.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000      155
8.284978 0.569956            2            2.0   1.0000   1.7550      207
5.384596 2.484213            4            4.0   0.0000   1.6859      249
3.444507 1.504419            8            8.0   1.0000   0.1637       45
110.780388 218.116268           16           16.0   3.0000   7.7153      139
75.235660 39.690933           32           32.0   2.0000   4.0961      151
51.536644 27.837627           64           64.0   2.0000   1.9416       7

In [25]:
! vw --testonly --initial_regressor drugs.model.bin --predictions drugs.preductions.txt test.vw

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
predictions = drugs.preductions.txt
Num weight bits = 23
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.447677 0.447677            1            1.0   2.0000   2.6691      317
2.223839 4.000000            2            2.0   2.0000   0.0000      237
4.300955 6.378071            4            4.0  10.0000  11.9381      163
25.360267 46.419579            8            8.0   1.0000  14.3987       59
17.799924 10.239580           16           16.0   2.0000   9.0071      285
24.017554 30.235185           32           32.0   7.0000   3.3096      125
158.437527 292.857500           64           64.0  10.0000  44.5076      175
336.991100 515.544673          128     

In [26]:
calc_r2('drugs.preductions.txt', 'test.vw')

0.6526252786437012

Обучили модель на **0.65** за **30** секунд.

Посмотрим, что будет если мы обучим модель только на части данных

In [27]:
%%time

! vw --final_regressor drugs.model.bin train.part1.vw \
    --onethread \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
final_regressor = drugs.model.bin
Num weight bits = 23
learning rate = 20
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.part1.vw.cache
Reading datafile = train.part1.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.000000 16.000000            1            1.0   4.0000   0.0000      155
8.284978 0.569956            2            2.0   1.0000   1.7550      207
5.384596 2.484213            4            4.0   0.0000   1.6859      249
3.444507 1.504419            8            8.0   1.0000   0.1637       45
110.780388 218.116268           16           16.0   3.0000   7.7153      139
75.235660 39.690933           32           32.0   2.0000   4.0961      151
51.536644 27.837627           64           64.0   2.0000   1.9416      

In [28]:
! vw --testonly --initial_regressor drugs.model.bin --predictions drugs.preductions.txt test.vw

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
predictions = drugs.preductions.txt
Num weight bits = 23
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1054.792969 1054.792969            1            1.0   2.0000  34.4776      317
529.396484 4.000000            2            2.0   2.0000   0.0000      237
351.250557 173.104630            4            4.0  10.0000  28.3633      163
214.487823 77.725090            8            8.0   1.0000  16.5387       59
140.830627 67.173430           16           16.0   2.0000  21.5064      285
101.366076 61.901526           32           32.0   7.0000   3.7109      125
202.674485 303.982894           64           64.0  10.0000  32.8632      175
677.614774 1152.555063  

In [29]:

calc_r2('drugs.preductions.txt', 'test.vw')

0.38178390321607647

Гораздо быстрее обучились, но потеряли в качестве. 

Модель на **0.38** за **6** секунд

**Мораль** - семплирование не самых удачный подход, чтобы получать качество, нужно засовывать в модель вообще все данные.

Запустим в фоновом режиме `spanning_tree` и проверим что он правда работает.

Далее воркеры будут подключаться к нему по tcp.

In [30]:
%%bash --bg --out OUT --err ERR
spanning_tree --nondaemon

In [31]:
! ps aux | grep spanning_tree

ubuntu      7380  0.0  0.0   6068  1612 ?        S    15:44   0:00 spanning_tree --nondaemon
ubuntu      7383  0.0  0.0   9492  3264 pts/0    Ss+  15:44   0:00 /bin/bash -c  ps aux | grep spanning_tree


Пора запускать рабочих. Для этого используется уже известная команда vw, в которую просто добавляются специальные параметры

* `--span_server` - указываем адрес, где находится менеджер (spanning_tree). В нашем случае это localhost. В реальной жизни там мог бы быть IP адрес другой машины
* `--unique_id` - так как один spanning_tree может обрабатывать сразу много различных процессов обучения, то необходимо их как-то разграничить. Для этого используется unique_id - это число, которое должно быть одинаковым для всех ваших рабочих, чтобы их не перепутали с другими. Например ваш коллега также обучает VW но для другой задачи - он может подключить свои VW к этому же spanning_tree указав для них unique_id = 0. В таком случае вам, чтобы подключиться, нужно запускать свои рабочие например с unique_id = 5, чтобы они не смешались с рабочими вашего коллеги.
* `--total` - число рабочих, которое вы планируете подключить в текущей сессии обучения
* `--node` - идентификатор текущего рабочего. Нумерация начинается с нуля, поэтому если вы хотите запустить 3 рабочих, то им нужно выдать значения для --node 0, 1 и 2.
* `-d` - данные для обработки для текущего рабочего
Все остальные параметры обучения должны быть одинаковыми для всех рабочих.

Чтобы сохранить коэффициенты полученной модели, необходимо для какого-то одного рабочего указать через `-f` или `--final_regressor` файл, куда записать результат. Точно также, как мы это делали в предыдущей лабораторной.

Запустим двух рабочих. Первого запустим также в фоне, а вот второй запустим прямо в ноутбуке и будем следить за процессом обучения.

In [32]:
%%bash --bg --out OUT --err ERR

vw -d train.part1.vw \
    --span_server localhost \
    --total 2 \
    --node 0 \
    --unique_id 1 \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k

In [33]:
! ps aux | grep vw

ubuntu      7509  0.0  0.8 222760 133016 ?       Rl   15:49   0:00 vw -d train.part1.vw --span_server localhost --total 2 --node 0 --unique_id 1 --learning_rate 20.0 --bit_precision 23 --passes 40 --ngram r2 --interactions dc --cache -k
ubuntu      7513  0.0  0.0   9492  3264 pts/0    Ss+  15:49   0:00 /bin/bash -c  ps aux | grep vw
ubuntu      7515  0.0  0.0   8900   724 pts/0    S+   15:49   0:00 grep vw


In [34]:
%%time

! vw -d train.part2.vw \
    --span_server localhost \
    --total 2 \
    --node 1 \
    --unique_id 1 \
    --learning_rate 20.0 \
    --bit_precision 23 \
    --passes 40 \
    --ngram r2 \
    --interactions dc \
    --cache -k \
    -f drugs.model.bin

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
final_regressor = drugs.model.bin
Num weight bits = 23
learning rate = 20
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.part2.vw.cache
Reading datafile = train.part2.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
4.000000 4.000000            1            1.0   2.0000   0.0000      207
9.166723 14.333446            2            2.0   4.0000   0.2140       61
42.704962 76.243202            4            4.0   0.0000   6.1127      235
53.751544 64.798126            8            8.0  18.0000   2.1770      203
53.708574 53.665604           16           16.0  24.0000   6.9774      155
40.367511 27.026447           32           32.0   3.0000   6.5818      273
45.832557 51.297603           64           64.0   2.0000   8.2354     

In [35]:
! vw --testonly --initial_regressor drugs.model.bin --predictions drugs.preductions.txt test.vw

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
predictions = drugs.preductions.txt
Num weight bits = 23
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
2.849555 2.849555            1            1.0   2.0000   3.6881      317
3.424778 4.000000            2            2.0   2.0000   0.0000      237
6.356657 9.288537            4            4.0  10.0000  13.0947      163
34.073244 61.789832            8            8.0   1.0000  16.2863       59
32.563332 31.053420           16           16.0   2.0000  16.5415      285
31.656058 30.748784           32           32.0   7.0000   3.2724      125
82.980499 134.304941           64           64.0  10.0000  43.8365      175
333.861668 584.742837          128      

In [36]:
calc_r2('drugs.preductions.txt', 'test.vw')

0.6496397151076883

Качество получилось даже немного больше, чем при одиночном запуске.

Сильного ускорения по времени мы не увидели, потому что мы все это запускаем на одной машине. Однако если запускать эти воркеры на разных машинах и на больших объемах данных, то можно увидеть сильное ускорение процесса обучения.

И основное достижение этого алгоритма - теперь мы можем размещать данные по нескольким машинам, что позволяет нам теоретически обработать датасет произвольного размера.

### VW на Hadoop

VW достаточно несложно запустить в виде обычной MapReduce задачи. Для этого даже есть готовый скрипт, который написан авторами инструмента. 

Почитать про то, как запускать этот инструмент на Hadoop можно вот здесь - https://github.com/VowpalWabbit/vowpal_wabbit/tree/master/cluster .

Мы же с вами более внимательно рассмотрим более удобный интерфейс для распределенного обучения VW на кластере.

### SynapseML

Существует целый набор библиотек для Spark от Microsoft, который позволяет удобно и быстро запускать распределенные алгоритмы на кластере Spark. Про все возможности можно почитать на официальном GitHub - https://github.com/microsoft/SynapseML

Мы с вами воспользуемся двумя инструментами оттуда - VW и LightGBM (градиентный бустинг).


Чтобы поставить SynapseML в окружение с lyvi , достаточно просто переконфигурировать сессию спарка.

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
se = pyspark.sql.SparkSession.builder.appName("MyApp2") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.5") \
            .config("spark.dynamicAllocation.enabled", False) \
            .config("spark.locality.wait", 0) \
            .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13f4f7a4-c2f6-4223-b0fb-5865321bda65;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;0.9.5 in central
	found com.microsoft.azure#synapseml-core_2.12;0.9.5 in central
	found org.scalactic#

In [3]:
! cd /home/ubuntu/.ivy2/jars && \
    cp io.netty_netty-transport-native-epoll-4.1.68.Final-linux-x86_64.jar io.netty_netty-transport-native-epoll-4.1.68.Final.jar && \
    cp io.netty_netty-transport-native-kqueue-4.1.68.Final-osx-x86_64.jar io.netty_netty-transport-native-kqueue-4.1.68.Final.jar && \
    cp io.netty_netty-resolver-dns-native-macos-4.1.68.Final-osx-x86_64.jar io.netty_netty-resolver-dns-native-macos-4.1.68.Final.jar

In [4]:
from pyspark.sql.functions import when, col
from pyspark.ml import Pipeline
from synapse.ml.vw import VowpalWabbitFeaturizer, VowpalWabbitRegressor

In [5]:
data = se.read.option("delimiter", "\t").csv('/user/drugs/data/*', header=True, inferSchema=True)

In [6]:
data.limit(10).toPandas()

,_c0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""""""It has no side effect, I take it in combina...",9.0,"May 20, 2012",27.0
1,95260,Guanfacine,ADHD,"""""""My son is halfway through his fourth week o...",None,None,NaN
2,We have tried many different medications and s...,8.0,"April 27, 2010",192,None,None,NaN
3,92703,Lybrel,Birth Control,"""""""I used to take another oral contraceptive, ...",None,None,NaN
4,The positive side is that I didn&#039;t have a...,5.0,"December 14, 2009",17,None,None,NaN
5,138000,Ortho Evra,Birth Control,"""""""This is my first time using any form of bir...",8.0,"November 3, 2015",10.0
6,35696,Buprenorphine / naloxone,Opiate Dependence,"""""""Suboxone has completely turned my life arou...",9.0,"November 27, 2016",37.0
7,155963,Cialis,Benign Prostatic Hyperplasia,"""""""2nd day on 5mg started to work with rock ha...",2.0,"November 28, 2015",43.0
8,165907,Levonorgestrel,Emergency Contraception,"""""""He pulled out, but he cummed a bit in me. I...",1.0,"March 7, 2017",5.0
9,102654,Aripiprazole,Bipolar Disorde,"""""""Abilify changed my life. There is hope. I w...",10.0,"March 14, 2015",32.0


In [7]:
data.columns

['_c0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount']

In [8]:
columns = [
    '_c0',
    'd',
    'c',
    'r',
    'rating',
    'data',
    'target',
]
df = data.toDF(*columns)
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- d: string (nullable = true)
 |-- c: string (nullable = true)
 |-- r: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- data: string (nullable = true)
 |-- target: integer (nullable = true)



In [9]:
train, test = (
    df
    .na.drop('any')
    .randomSplit([0.9, 0.1], 422)
)

In [10]:
train.limit(20).toPandas()

,_c0,d,c,r,rating,data,target
0,10000,Lo Loestrin Fe,Birth Control,"""""""I was on this birth control for 8 months. T...",7.0,"April 10, 2013",4
1,100012,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;ve been taking Velivet for about a y...",9.0,"June 17, 2017",2
2,100013,Desogestrel / ethinyl estradiol,Birth Control,"""""""Gives me heartburn and indigestion. Also ma...",2.0,"June 8, 2017",1
3,100029,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was switched from Azurette to Viorele by ...",5.0,"December 6, 2017",0
4,10004,Lo Loestrin Fe,Birth Control,"""""""I&#039;m 41, using for cramps and excessive...",2.0,"March 31, 2013",12
5,100055,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on Apri for about a year. The first f...",5.0,"May 2, 2017",1
6,10007,Lo Loestrin Fe,Birth Control,"""""""I posted on this forum when I first started...",9.0,"March 10, 2013",18
7,100071,Desogestrel / ethinyl estradiol,Birth Control,"""""""So I&#039;ve only been on this pill for a m...",8.0,"March 7, 2017",7
8,100085,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on this for a year or two, I had been...",1.0,"January 28, 2017",4
9,100087,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;m 19 and have been on this birth con...",9.0,"January 26, 2017",3


Создадим объект для создания признаков в формате VW. Он принимает dataframe и возвращает dataframe но уже с новой колонкой, в которой записаны эти признаки

In [11]:
vw_featurizer = VowpalWabbitFeaturizer(
    inputCols=["rating"], 
    stringSplitInputCols=["d", "c", "r"],
    outputCol="features",
    numBits=24
)

In [12]:
x = vw_featurizer.transform(train).rdd.first()
x['features']

SparseVector(16777216, {139281: 1.0, 1016583: 1.0, 1102820: 1.0, 1162472: 2.0, 1505820: 1.0, 2204132: 1.0, 2339869: 1.0, 2673967: 1.0, 2679020: 1.0, 2839966: 1.0, 2991125: 1.0, 3257429: 1.0, 3346639: 6.0, 3410361: 1.0, 3446374: 1.0, 3783103: 2.0, 3803230: 1.0, 4415074: 1.0, 4597225: 1.0, 4709980: 1.0, 4778637: 1.0, 4890812: 1.0, 5367110: 1.0, 5426661: 2.0, 5481570: 1.0, 5728618: 1.0, 5837165: 1.0, 5881332: 1.0, 6192782: 1.0, 6362983: 1.0, 6366072: 1.0, 6737743: 1.0, 7337106: 2.0, 7608613: 1.0, 7636861: 3.0, 8148668: 1.0, 8336163: 1.0, 8515415: 1.0, 9170487: 1.0, 9519332: 1.0, 9651660: 1.0, 9787552: 1.0, 9845063: 1.0, 9894590: 1.0, 9970646: 1.0, 10090473: 1.0, 10189708: 1.0, 11318998: 1.0, 11946903: 1.0, 12243560: 1.0, 12463287: 2.0, 12730453: 1.0, 12741825: 1.0, 13350349: 1.0, 13357553: 2.0, 13735132: 3.0, 13901790: 1.0, 14380379: 1.0, 14524688: 1.0, 14608623: 1.0, 14946398: 1.0, 15174436: 1.0, 15384876: 1.0, 15847749: 1.0, 16681717: 1.0, 16772414: 1.0})

Создадим объект для обучения классификатора. Схема работы точно такая же - принимает на вход dataframe и потом может модифицировать другой dataframe, делая предсказание.

In [13]:
args = "--learning_rate 20.0 --bit_precision 24 --ngram r2 --interactions dc"
vw_model = VowpalWabbitRegressor(
    featuresCol="features",
    labelCol="target",
    args=args,
    numPasses=40
)

Соберем их в единый пайплайн

In [14]:
vw_pipeline = Pipeline(stages=[vw_featurizer, vw_model])

In [15]:
vw_trained = vw_pipeline.fit(train)

inbound connection from 10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795) serv=38608
10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795): nonce=839980628
10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795): total=4
10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795): node id=1
inbound connection from 10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795) serv=38616
10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795): nonce=839980628
10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795): total=4
10.128.0.37(rc1a-dataproc-d-z4c3crwrio0motqz.mdb.yandexcloud.net:37795): node id=0


nonce 839980628 still waiting for 3 nodes out of 4 for example node 0
nonce 839980628 still waiting for 2 nodes out of 4 for example node 2


inbound connection from 10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795) serv=44152
10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795): nonce=839980628
10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795): total=4
10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795): node id=3


nonce 839980628 still waiting for 1 nodes out of 4 for example node 2


inbound connection from 10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795) serv=44168
10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795): nonce=839980628
10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795): total=4
10.128.0.27(rc1a-dataproc-d-1jipfldazn1nw9u1.mdb.yandexcloud.net:37795): node id=2


In [16]:
prediction = vw_trained.transform(test)

Generating 2-grams for r namespaces.
creating features for following interactions: dc 
only testing
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


In [17]:
prediction.limit(10).toPandas()

,_c0,d,c,r,rating,data,target,features,rawPrediction,prediction
0,100009,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on reclipsen for less than 2 weeks an...",1.0,"June 28, 2017",2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000
1,10002,Lo Loestrin Fe,Birth Control,"""""""Well, I&#039;ve been on this right now for ...",6.0,"April 4, 2013",10,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20.970985,20.970985
2,100091,Desogestrel / ethinyl estradiol,Birth Control,"""""""I started Apri a year and 4 months ago. Whe...",8.0,"January 8, 2017",3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.158253,2.158253
3,100094,Desogestrel / ethinyl estradiol,Birth Control,"""""""I started Apri after switching from a birth...",8.0,"December 29, 2016",2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000
4,1001,Everolimus,Breast Cance,"""""""Stage 4 with lymph and bone bone metastasis...",10.0,"August 31, 2015",16,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43.083103,43.083103
5,100217,Desogestrel / ethinyl estradiol,Birth Control,"""""""Horrible experience! Has never had trouble ...",1.0,"October 26, 2015",3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000
6,100242,Desogestrel / ethinyl estradiol,Birth Control,"""""""I first started with Reclipsen a year ago f...",10.0,"December 28, 2015",1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000
7,100259,Desogestrel / ethinyl estradiol,Birth Control,"""""""I changed to this birth control at the advi...",4.0,"September 21, 2015",2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",22.114550,22.114550
8,100331,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;ve been on this pill for almost a ye...",9.0,"October 19, 2016",4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.716258,9.716258
9,100337,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;d say this is a good/decent pill ove...",9.0,"October 3, 2016",1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5.560011,5.560011


In [18]:
from synapse.ml.train import ComputeModelStatistics
metrics = ComputeModelStatistics(
    evaluationMetric='regression',
    labelCol='target',
    scoresCol='prediction'
).transform(prediction)

In [19]:
metrics.toPandas()

,mean_squared_error,root_mean_squared_error,R^2,mean_absolute_error
0,684.964677,26.17183,0.475109,16.509442


### SparkML

Нужно отметить, что в стандартной библиотеке Spark присутствует модуль для машинного обучения.

**ОДНАКО** нужно сказать, что работает он крайне плохо. Лучшее, что вы можете с ним сделать - это попробовать один раз его запустить и понять, что больше никогда не будете его использовать.

Это правда важно, потому что это не звучит слишком убедительно, что стандартная библиотека для ML насколько уж плохо работет и наверное все таки есть случаи, когда она работает хорошо, правда ведь? Ответ - вполне возможно. Чтобы вам самим понять, есть ли такие случаи, попробуйте самостоятельно что-то обучить на SparkML и прочувствуйте границы применимости :)

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
sc = pyspark.SparkContext(appName="lsml-app-1")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-21 16:33:08,874 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
2023-02-21 16:33:13,977 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [3]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [4]:
from pyspark.sql import SparkSession, Row

se = SparkSession(sc)

In [5]:
data = se.read.option("delimiter", "\t").csv('/user/drugs/data/*', header=True, inferSchema=True)

In [6]:
data = (
    data
    .na.drop('any')
    .withColumn('ratingNum', data.rating.cast('integer'))
)


train, test = data.randomSplit([0.9, 0.1], 422)
train, test = train.cache(), test.cache()

In [8]:
tokenizer = Tokenizer(inputCol="review", outputCol="words")
wordsData = tokenizer.transform(train)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2**23)
featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

rescaledData = idfModel.transform(featurizedData)

In [9]:
rescaledData.limit(5).toPandas()

2023-02-21 16:34:12,611 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 128.1 MiB


,_c0,drugName,condition,review,rating,date,usefulCount,ratingNum,words,rawFeatures,features
0,10,Medroxyprogesterone,Abnormal Uterine Bleeding,"""""""I&#039;m 17 years old and I got shot in Aug...",7.0,"October 20, 2015",2,7,"[""""""i&#039;m, 17, years, old, and, i, got, sho...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1000,Everolimus,Breast Cance,"""""""Although the medication did effectively tre...",2.0,"March 15, 2016",4,2,"[""""""although, the, medication, did, effectivel...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,10000,Lo Loestrin Fe,Birth Control,"""""""I was on this birth control for 8 months. T...",7.0,"April 10, 2013",4,7,"[""""""i, was, on, this, birth, control, for, 8, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,100004,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have been taking Azurette for 3 years now...",8.0,"July 11, 2017",1,8,"[""""""i, have, been, taking, azurette, for, 3, y...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,100007,Desogestrel / ethinyl estradiol,Birth Control,"""""""At the beginning, Kariva seemed to be worki...",5.0,"June 29, 2017",0,5,"[""""""at, the, beginning,, kariva, seemed, to, b...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
stringIndexer = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoder = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

pipeline = Pipeline(stages=[stringIndexer, encoder])
ohe = pipeline.fit(rescaledData).transform(rescaledData)

In [11]:
x = ohe.limit(1).rdd.first()
x

2023-02-21 16:34:15,690 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 128.4 MiB
2023-02-21 16:34:16,777 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 128.3 MiB


Row(_c0='10', drugName='Medroxyprogesterone', condition='Abnormal Uterine Bleeding', review='"""I&#039;m 17 years old and I got shot in August 2015, personally. I don&#039;t mind it. I mean, I bleed little bits and random times, but I&#039;d rather have the blood that&#039;s supposed to come out, come out and not worry about where it&#039;s going or staying in my body. I have my other injection in November on the 2nd, and I&#039;m still wondering if I could take it again. The only downside to the injection is that I gained access weight and I&#039;m kind of moody."""', rating='7.0', date='October 20, 2015', usefulCount=2, ratingNum=7, words=['"""i&#039;m', '17', 'years', 'old', 'and', 'i', 'got', 'shot', 'in', 'august', '2015,', 'personally.', 'i', 'don&#039;t', 'mind', 'it.', 'i', 'mean,', 'i', 'bleed', 'little', 'bits', 'and', 'random', 'times,', 'but', 'i&#039;d', 'rather', 'have', 'the', 'blood', 'that&#039;s', 'supposed', 'to', 'come', 'out,', 'come', 'out', 'and', 'not', 'worry',

In [12]:
x['drugVec']

SparseVector(3492, {13: 1.0})

Подготавливаем признаки

In [17]:
wordsData = tokenizer.transform(train)

tokenizer = Tokenizer(inputCol="review", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2**10)
idf = IDF(inputCol="rawFeatures", outputCol="revviewFeatures")

stringIndexerCondition = StringIndexer(inputCol='condition', outputCol = "conditionIndex").setHandleInvalid("skip")
encoderCondition = OneHotEncoder(inputCol="conditionIndex", outputCol="conditionVec")

stringIndexerDrug = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoderDrug = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

assembler = VectorAssembler(inputCols=["drugVec", "conditionVec", "revviewFeatures", 'ratingNum'], outputCol="features")

preproc = Pipeline(stages=[
    tokenizer,
    hashingTF,
    idf,
    stringIndexerCondition,
    encoderCondition,
    stringIndexerDrug,
    encoderDrug,
    assembler
])

In [14]:
train_proc = preproc.fit(train).transform(train).cache()

In [16]:
train_proc.limit(10).toPandas()

,_c0,drugName,condition,review,rating,date,usefulCount,ratingNum,words,rawFeatures,revviewFeatures,conditionIndex,conditionVec,drugIndex,drugVec,features
0,10,Medroxyprogesterone,Abnormal Uterine Bleeding,"""""""I&#039;m 17 years old and I got shot in Aug...",7.0,"October 20, 2015",2,7,"[""""""i&#039;m, 17, years, old, and, i, got, sho...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",13.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1000,Everolimus,Breast Cance,"""""""Although the medication did effectively tre...",2.0,"March 15, 2016",4,2,"[""""""although, the, medication, did, effectivel...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",83.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1322.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,10000,Lo Loestrin Fe,Birth Control,"""""""I was on this birth control for 8 months. T...",7.0,"April 10, 2013",4,7,"[""""""i, was, on, this, birth, control, for, 8, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",35.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,100004,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have been taking Azurette for 3 years now...",8.0,"July 11, 2017",1,8,"[""""""i, have, been, taking, azurette, for, 3, y...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,100007,Desogestrel / ethinyl estradiol,Birth Control,"""""""At the beginning, Kariva seemed to be worki...",5.0,"June 29, 2017",0,5,"[""""""at, the, beginning,, kariva, seemed, to, b...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,100008,Desogestrel / ethinyl estradiol,Birth Control,"""""""This is yet another update. Just finished m...",1.0,"June 28, 2017",0,1,"[""""""this, is, yet, another, update., just, fin...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,100009,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on reclipsen for less than 2 weeks an...",1.0,"June 28, 2017",2,1,"[""""""i, was, on, reclipsen, for, less, than, 2,...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3.3244338839699816, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,100011,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have been on Apri for 4 months, I&#039;m ...",8.0,"June 19, 2017",3,8,"[""""""i, have, been, on, apri, for, 4, months,, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,100012,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;ve been taking Velivet for about a y...",9.0,"June 17, 2017",2,9,"[""""""i&#039;ve, been, taking, velivet, for, abo...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
tokenizer = Tokenizer(inputCol="review", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2**23)
idf = IDF(inputCol="rawFeatures", outputCol="revviewFeatures")

preproc = Pipeline(stages=[
    tokenizer,
    hashingTF,
    idf
])

Запустить сбор признаков, который написан, у вас скорее всего не получится. Поэтому попробуем урезать количество вычислений - может быть получится.

In [12]:
stringIndexerCondition = StringIndexer(inputCol='condition', outputCol = "conditionIndex").setHandleInvalid("skip")
encoderCondition = OneHotEncoder(inputCol="conditionIndex", outputCol="conditionVec")

stringIndexerDrug = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoderDrug = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

assembler = VectorAssembler(inputCols=["drugVec", "conditionVec", 'ratingNum'], outputCol="features")

preproc = Pipeline(stages=[
    stringIndexerCondition,
    encoderCondition,
    stringIndexerDrug,
    encoderDrug,
    assembler
])

In [18]:
preproc = preproc.fit(data)

In [19]:
train_proc = preproc.transform(train).cache()
test_proc = preproc.transform(test).cache()

In [20]:
train_proc.limit(10).toPandas()

,_c0,drugName,condition,review,rating,date,usefulCount,ratingNum,words,rawFeatures,revviewFeatures,conditionIndex,conditionVec,drugIndex,drugVec,features
0,10,Medroxyprogesterone,Abnormal Uterine Bleeding,"""""""I&#039;m 17 years old and I got shot in Aug...",7.0,"October 20, 2015",2,7,"[""""""i&#039;m, 17, years, old, and, i, got, sho...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1000,Everolimus,Breast Cance,"""""""Although the medication did effectively tre...",2.0,"March 15, 2016",4,2,"[""""""although, the, medication, did, effectivel...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",83.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1379.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,10000,Lo Loestrin Fe,Birth Control,"""""""I was on this birth control for 8 months. T...",7.0,"April 10, 2013",4,7,"[""""""i, was, on, this, birth, control, for, 8, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",35.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,100004,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have been taking Azurette for 3 years now...",8.0,"July 11, 2017",1,8,"[""""""i, have, been, taking, azurette, for, 3, y...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,100007,Desogestrel / ethinyl estradiol,Birth Control,"""""""At the beginning, Kariva seemed to be worki...",5.0,"June 29, 2017",0,5,"[""""""at, the, beginning,, kariva, seemed, to, b...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,100008,Desogestrel / ethinyl estradiol,Birth Control,"""""""This is yet another update. Just finished m...",1.0,"June 28, 2017",0,1,"[""""""this, is, yet, another, update., just, fin...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,100009,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on reclipsen for less than 2 weeks an...",1.0,"June 28, 2017",2,1,"[""""""i, was, on, reclipsen, for, less, than, 2,...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(3.326994633259291, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,100011,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have been on Apri for 4 months, I&#039;m ...",8.0,"June 19, 2017",3,8,"[""""""i, have, been, on, apri, for, 4, months,, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,100012,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;ve been taking Velivet for about a y...",9.0,"June 17, 2017",2,9,"[""""""i&#039;ve, been, taking, velivet, for, abo...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [21]:
train_proc.rdd.first()

Row(_c0='10', drugName='Medroxyprogesterone', condition='Abnormal Uterine Bleeding', review='"""I&#039;m 17 years old and I got shot in August 2015, personally. I don&#039;t mind it. I mean, I bleed little bits and random times, but I&#039;d rather have the blood that&#039;s supposed to come out, come out and not worry about where it&#039;s going or staying in my body. I have my other injection in November on the 2nd, and I&#039;m still wondering if I could take it again. The only downside to the injection is that I gained access weight and I&#039;m kind of moody."""', rating='7.0', date='October 20, 2015', usefulCount=2, ratingNum=7, words=['"""i&#039;m', '17', 'years', 'old', 'and', 'i', 'got', 'shot', 'in', 'august', '2015,', 'personally.', 'i', 'don&#039;t', 'mind', 'it.', 'i', 'mean,', 'i', 'bleed', 'little', 'bits', 'and', 'random', 'times,', 'but', 'i&#039;d', 'rather', 'have', 'the', 'blood', 'that&#039;s', 'supposed', 'to', 'come', 'out,', 'come', 'out', 'and', 'not', 'worry',

Если все таки удалось собрать датасет, то запускаем линейную регрессию

In [22]:
lr = LinearRegression(featuresCol='features', labelCol='usefulCount', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [23]:
lrModel = lr.fit(train_proc)

2023-02-21 16:37:09,275 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2023-02-21 16:37:09,276 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [24]:
lrModel.coefficients

SparseVector(5493, {0: -3.7545, 1: -0.0355, 7: 0.8078, 8: -2.7885, 11: 18.5616, 12: -2.3546, 13: 4.6498, 15: -4.9653, 17: 0.6126, 18: 2.3493, 19: -3.2576, 20: -1.7976, 22: -1.3357, 24: 1.5977, 25: 5.2494, 27: -5.5526, 28: 15.0816, 30: -3.3315, 31: -5.9904, 32: 3.7087, 34: -1.4971, 36: 16.8379, 37: -3.4556, 42: -4.2676, 43: 9.7683, 44: 17.9128, 45: 6.7376, 46: -5.9377, 47: 19.7565, 49: 1.1777, 51: 21.3983, 52: -0.2147, 54: -3.9873, 59: 4.3384, 61: -6.6768, 62: 0.3845, 63: 15.678, 64: 1.9675, 68: -6.2524, 69: 5.5691, 73: 2.931, 74: 21.2718, 75: 11.3858, 77: 12.8736, 82: 1.9036, 83: 12.0439, 85: 0.8322, 88: -1.225, 89: -0.7137, 91: 3.5373, 94: -3.2439, 97: 8.8043, 101: 4.8084, 104: 8.5891, 106: 13.7059, 107: -4.3021, 109: 12.598, 110: 20.9191, 111: 8.42, 113: -5.6995, 116: 0.6798, 118: 14.4833, 119: 2.3975, 120: 6.5829, 122: 8.1533, 123: 12.0032, 124: 23.0656, 132: 0.2559, 140: 6.9898, 143: 35.3707, 145: 12.4007, 146: 14.5449, 149: 5.9645, 152: 3.6349, 153: 8.696, 157: 2.7769, 158: 0.9476

In [25]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = lrModel.transform(test_proc)

In [26]:
predictions.limit(10).toPandas()

,_c0,drugName,condition,review,rating,date,usefulCount,ratingNum,words,rawFeatures,revviewFeatures,conditionIndex,conditionVec,drugIndex,drugVec,features,prediction
0,100,Medroxyprogesterone,Birth Control,"""""""Depo was not for me, but that does not mean...",5.0,"August 17, 2015",2,5,"[""""""depo, was, not, for, me,, but, that, does,...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.807138
1,100002,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have been taking this birth control for f...",4.0,"July 12, 2017",2,4,"[""""""i, have, been, taking, this, birth, contro...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.221327
2,100017,Desogestrel / ethinyl estradiol,Birth Control,"""""""Love it! Been continuously dosing without b...",9.0,"May 30, 2017",3,9,"[""""""love, it!, been, continuously, dosing, wit...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",13.877902
3,10002,Lo Loestrin Fe,Birth Control,"""""""Well, I&#039;ve been on this right now for ...",6.0,"April 4, 2013",10,6,"[""""""well,, i&#039;ve, been, on, this, right, n...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",35.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",11.811320
4,100021,Desogestrel / ethinyl estradiol,Birth Control,"""""""So I&#039;ve been on this birth control a l...",5.0,"May 18, 2017",3,5,"[""""""so, i&#039;ve, been, on, this, birth, cont...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.463578
5,100107,Desogestrel / ethinyl estradiol,Birth Control,"""""""completely lost every bit of sex drive i ha...",5.0,"November 23, 2016",1,5,"[""""""completely, lost, every, bit, of, sex, dri...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.795306
6,100123,Desogestrel / ethinyl estradiol,Birth Control,"""""""I haven&#039;t taken any BC in 5 years sinc...",1.0,"November 2, 2016",2,1,"[""""""i, haven&#039;t, taken, any, bc, in, 5, ye...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.579616
7,100126,Desogestrel / ethinyl estradiol,Birth Control,"""""""I have used Cerelle for 9 days I feel very...",10.0,"October 28, 2016",1,10,"[""""""i, have, , used, cerelle, for, 9, days, i,...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14.858223
8,100165,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;ve had a pretty good experience with...",8.0,"Ma

In [27]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="usefulCount", metricName="r2")
lr_evaluator.evaluate(predictions)

0.2997379555791938

С недавнего времени на Spark появился CatBoost. Давайте попробуем поиграться с этим инструментом.

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
se = pyspark.sql.SparkSession.builder.appName("MyApp2") \
            .config("spark.jars.packages", "ai.catboost:catboost-spark_3.0_2.12:1.1.1") \
            .config("spark.dynamicAllocation.enabled", False) \
            .config("spark.locality.wait", 0) \
            .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
ai.catboost#catboost-spark_3.0_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fb96a777-13b6-433d-bae6-2ce4ee508609;1.0
	confs: [default]
	found ai.catboost#catboost-spark_3.0_2.12;1.1.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.6.0 in central
	found 

In [3]:
import catboost_spark

In [4]:
data = se.read.option("delimiter", "\t").csv('/user/drugs/data/*', header=True, inferSchema=True)

data = (
    data
    .na.drop('any')
    .withColumn('ratingNum', data.rating.cast('integer'))
)


train, test = data.randomSplit([0.9, 0.1], 422)
train, test = train.cache(), test.cache()

In [5]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [6]:
stringIndexerCondition = StringIndexer(inputCol='condition', outputCol = "conditionIndex").setHandleInvalid("skip")
encoderCondition = OneHotEncoder(inputCol="conditionIndex", outputCol="conditionVec")

stringIndexerDrug = StringIndexer(inputCol='drugName', outputCol = "drugIndex").setHandleInvalid("skip")
encoderDrug = OneHotEncoder(inputCol="drugIndex", outputCol="drugVec")

assembler = VectorAssembler(inputCols=["drugVec", "conditionVec", 'ratingNum'], outputCol="features")


preproc = Pipeline(stages=[
    stringIndexerCondition,
    encoderCondition,
    stringIndexerDrug,
    encoderDrug,
    assembler
])

In [7]:
preproc = preproc.fit(data)

In [8]:
train_proc = preproc.transform(train).cache()
test_proc = preproc.transform(test).cache()

In [9]:
train_proc.limit(10).toPandas()

,_c0,drugName,condition,review,rating,date,usefulCount,ratingNum,conditionIndex,conditionVec,drugIndex,drugVec,features
0,10000,Lo Loestrin Fe,Birth Control,"""""""I was on this birth control for 8 months. T...",7.0,"April 10, 2013",4,7,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",35.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,100012,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;ve been taking Velivet for about a y...",9.0,"June 17, 2017",2,9,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,100013,Desogestrel / ethinyl estradiol,Birth Control,"""""""Gives me heartburn and indigestion. Also ma...",2.0,"June 8, 2017",1,2,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,100029,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was switched from Azurette to Viorele by ...",5.0,"December 6, 2017",0,5,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,10004,Lo Loestrin Fe,Birth Control,"""""""I&#039;m 41, using for cramps and excessive...",2.0,"March 31, 2013",12,2,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",35.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,100055,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on Apri for about a year. The first f...",5.0,"May 2, 2017",1,5,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,10007,Lo Loestrin Fe,Birth Control,"""""""I posted on this forum when I first started...",9.0,"March 10, 2013",18,9,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",35.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,100071,Desogestrel / ethinyl estradiol,Birth Control,"""""""So I&#039;ve only been on this pill for a m...",8.0,"March 7, 2017",7,8,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,100085,Desogestrel / ethinyl estradiol,Birth Control,"""""""I was on this for a year or two, I had been...",1.0,"January 28, 2017",4,1,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,100087,Desogestrel / ethinyl estradiol,Birth Control,"""""""I&#039;m 19 and have been on this birth con...",9.0,"January 26, 2017",3,9,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",60.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import Row,SparkSession

In [11]:
srcDataSchema = [
    StructField("features", VectorUDT()),
    StructField("label", DoubleType())
]

In [12]:
train_proc.rdd.map(lambda x: Row(x.features, float(x.usefulCount))).take(1)

[<Row(SparseVector(4469, {35: 1.0, 3572: 1.0, 4468: 7.0}), 4.0)>]

In [14]:
trainData = train_proc.rdd.map(lambda x: Row(x.features, float(x.usefulCount)))

In [15]:
trainDf = se.createDataFrame(trainData, StructType(srcDataSchema))

In [16]:
trainDf.limit(10).toPandas()

,features,label
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12.0
5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
6,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",18.0
7,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",7.0
8,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0
9,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0


In [17]:
evalData = test_proc.rdd.map(lambda x: Row(x.features, float(x.usefulCount)))

In [18]:
evalDf = se.createDataFrame(evalData, StructType(srcDataSchema))

In [19]:
evalDf.limit(10).toPandas()

,features,label
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16.0
5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0
6,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
7,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0
8,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0
9,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0


In [20]:
trainPool = catboost_spark.Pool(trainDf)
evalPool = catboost_spark.Pool(evalDf)

In [21]:
regressor = catboost_spark.CatBoostRegressor()

In [22]:
model = regressor.fit(trainPool, evalDatasets=[evalPool])

2023-02-21 16:32:08,305 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1130.6 KiB
2023-02-21 16:32:08,383 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1130.6 KiB


KeyboardInterrupt: 

In [ ]:
predictions = model.transform(evalPool.data)
predictions.show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="r2")
lr_evaluator.evaluate(predictions)

In [ ]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="rmse")
lr_evaluator.evaluate(predictions)